# Chapter 12. Serial Correlation and Heteroskedasticity in Time Series Regressions 
[Home](http://solomonegash.com/) | [Stata](http://solomonegash.com/woodridge1/index.html) | [R](http://solomonegash.com/econometrics/rbook1/index.html)


In [1]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

from wooldridge import *

### Example 12.1. Testing for AR(1) Serial Correlation in the Phillips Curve

In [2]:
df = dataWoo("phillips")
df = df[(df['year']<1997)]

df['uhat1'] = smf.ols('df.inf ~ unem + 1', data=df).fit().resid
print(smf.ols('uhat1 ~ uhat1.shift(1)', data=df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  uhat1   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     24.34
Date:                Tue, 02 Jul 2024   Prob (F-statistic):           1.10e-05
Time:                        18:08:46   Log-Likelihood:                -110.88
No. Observations:                  48   AIC:                             225.8
Df Residuals:                      46   BIC:                             229.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.1134      0.359     -0.

In [3]:
df['uhat2'] = smf.ols('df.cinf ~ unem', data=df).fit().resid
print(smf.ols('uhat2 ~ uhat2.shift(1)', data=df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  uhat2   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                   0.08254
Date:                Tue, 02 Jul 2024   Prob (F-statistic):              0.775
Time:                        18:08:46   Log-Likelihood:                -99.620
No. Observations:                  47   AIC:                             203.2
Df Residuals:                      45   BIC:                             206.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.1942      0.300      0.

### Example 12.2. Testing for AR(1) Serial Correlation in the Minimum Wage Equation

In [4]:
df = dataWoo("prminwge")

df['uhat'] = smf.ols('lprepop ~ lmincov + lprgnp + lusgnp + t', data=df).fit().resid
AR1c = smf.ols('uhat ~  lmincov + lprgnp + lusgnp + t + uhat.shift(1)' , data=df).fit()
AR1s  = smf.ols('uhat ~ uhat.shift(1)', data=df).fit()

print(summary_col([AR1c, AR1s],stars=True,float_format='%0.3f',
                  model_names=['AR1c\n(b/se)','AR1s\n(b/se)'],
                 info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared), 
                           'Adj.R2':lambda x: "{:.3f}".format(x.rsquared_adj)}))


                 AR1c     AR1s 
                (b/se)   (b/se)
-------------------------------
Intercept      -0.851   -0.001 
               (1.093)  (0.004)
R-squared      0.242    0.165  
R-squared Adj. 0.120    0.141  
lmincov        0.038           
               (0.035)         
lprgnp         -0.078          
               (0.071)         
lusgnp         0.204           
               (0.195)         
t              -0.003          
               (0.004)         
uhat.shift(1)  0.481*** 0.417**
               (0.166)  (0.159)
N              37       37     
R2             0.242    0.165  
Adj.R2         0.120    0.141  
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Example 12.3. Testing for AR(3) Serial Correlation

In [5]:
df = dataWoo("barium")
df['u'] = smf.ols('lchnimp ~ lchempi + lgas + lrtwex + befile6 + affile6 + afdec6', data=df).fit().resid
AR3 = smf.ols('u ~ lchempi + lgas + lrtwex + befile6 + affile6 + afdec6 + u.shift(1) + u.shift(2) + u.shift(3) + 1', data = df).fit()
print(AR3.summary())

                            OLS Regression Results                            
Dep. Variable:                      u   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1.719
Date:                Tue, 02 Jul 2024   Prob (F-statistic):             0.0920
Time:                        18:08:46   Log-Likelihood:                -104.56
No. Observations:                 128   AIC:                             229.1
Df Residuals:                     118   BIC:                             257.6
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -14.3692     20.656     -0.696      0.4

In [6]:
hypotheses = '(u.shift(1) = u.shift(2) = u.shift(3) = 0)'
f_test = AR3.f_test(hypotheses)
print(f_test)

<F test: F=5.1229144202079056, p=0.002289781725887361, df_denom=118, df_num=3>


### Example 12.4. Prais-Winsten Estimation in the Event Study

In [7]:
df = dataWoo("barium")
OLS = smf.ols('lchnimp ~ lchempi + lgas + lrtwex + befile6 + affile6 + afdec6', data=df).fit()

In [8]:
def ols_ar1(model,rho,drop1=True):
    x = model.model.exog
    y = model.model.endog
    ystar = y[1:]-rho*y[:-1]
    xstar = x[1:,]-rho*x[:-1,]
    if drop1 == False:
        ystar = np.append(np.sqrt(1-rho**2)*y[0],ystar)
        xstar = np.append([np.sqrt(1-rho**2)*x[0,]],xstar,axis=0)
    model_ar1 = sm.OLS(ystar,xstar).fit()
    return(model_ar1)

In [9]:
def OLSAR1(model,drop1=True):
    x = model.model.exog
    y = model.model.endog
    e = y-x@model.params
    e1 = e[:-1]; e0 = e[1:]
    rho0 = np.dot(e1,e[1:])/np.dot(e1,e1)
    rdiff = 1.0
    while(rdiff>1.0e-5):
        model1 = ols_ar1(model,rho0,drop1)
        e = y - (x @ model1.params)
        e1 = e[:-1]; e0 = e[1:]
        rho1 = np.dot(e1,e[1:])/np.dot(e1,e1)
        rdiff = np.sqrt((rho1-rho0)**2)
        rho0 = rho1
        print('Rho = ', rho0)
    model1 = ols_ar1(model,rho0,drop1)
    return(model1)

In [10]:
ar1_pw = OLSAR1(OLS ,drop1=False)
print(ar1_pw.summary())

Rho =  0.2910357321865985
Rho =  0.29300215914928224
Rho =  0.29319585917068286
Rho =  0.2932149681976122
Rho =  0.29321685363481764
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.984
Model:                            OLS   Adj. R-squared (uncentered):              0.983
Method:                 Least Squares   F-statistic:                              1096.
Date:                Tue, 02 Jul 2024   Prob (F-statistic):                   2.99e-108
Time:                        18:08:46   Log-Likelihood:                         -109.41
No. Observations:                 131   AIC:                                      232.8
Df Residuals:                     124   BIC:                                      252.9
Df Model:                           7                                                  
Covariance Type:            nonrobust                                      

### Example 12.5. Static Phillips Curve

In [11]:
df = dataWoo("phillips")
df = df[(df['year']<1997)]
ols = smf.ols('df.inf ~ unem', data=df).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 df.inf   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     2.616
Date:                Tue, 02 Jul 2024   Prob (F-statistic):              0.112
Time:                        18:08:46   Log-Likelihood:                -124.43
No. Observations:                  49   AIC:                             252.9
Df Residuals:                      47   BIC:                             256.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4236      1.719      0.828      0.4

In [12]:
ar1_pw = OLSAR1(ols ,drop1=False)
print(ar1_pw.summary())

Rho =  0.7306596839389279
Rho =  0.7719454382272319
Rho =  0.7792180497300137
Rho =  0.7803441605388693
Rho =  0.780514454709218
Rho =  0.7805401123896687
Rho =  0.7805439759940054
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.241
Model:                            OLS   Adj. R-squared (uncentered):              0.208
Method:                 Least Squares   F-statistic:                              7.446
Date:                Tue, 02 Jul 2024   Prob (F-statistic):                     0.00155
Time:                        18:08:46   Log-Likelihood:                         -108.62
No. Observations:                  49   AIC:                                      221.2
Df Residuals:                      47   BIC:                                      225.0
Df Model:                           2                                                  
Covariance Type:           

### Example 12.6. Differencing the Interest Rate Equation

In [13]:
df = dataWoo("intdef")
def1 = df['def']
ureg = smf.ols('i3 ~ df.inf + def1', data=df).fit()
df['u'] = ureg.resid
AR1u = smf.ols('u ~ u.shift(1)', data=df).fit()
ereg = smf.ols('ci3 ~ df.cinf + cdef', data=df).fit()
df['e'] = ereg.resid
AR1e = smf.ols('e ~ e.shift(1)', data=df).fit()

print(summary_col([AR1u, AR1e, ureg, ereg],stars=True,float_format='%0.3f',
                  model_names=['AR1u\n(b/se)','AR1e\n(b/se)', 'ureg\n(b/se)','ereg\n(b/se)'],
                 info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared), 
                           'Adj.R2':lambda x: "{:.3f}".format(x.rsquared_adj)}))


                 AR1u     AR1e    ureg     ereg 
                (b/se)   (b/se)  (b/se)   (b/se)
------------------------------------------------
Intercept      0.015    -0.041  1.733*** 0.042  
               (0.190)  (0.166) (0.432)  (0.171)
R-squared      0.377    0.005   0.602    0.176  
R-squared Adj. 0.366    -0.014  0.587    0.145  
cdef                                     -0.181 
                                         (0.148)
def1                            0.513***        
                                (0.118)         
df.cinf                                  0.149  
                                         (0.092)
df.inf                          0.606***        
                                (0.082)         
e.shift(1)              0.072                   
                        (0.134)                 
u.shift(1)     0.623***                         
               (0.110)                          
N              55       54      56       55     
R2             0.37

### Example 12.7. The Puerto Rican Minimum Wage

In [14]:
df = dataWoo("prminwge")
OLS2 =smf.ols('lprepop ~ lmincov + lprgnp + lusgnp + t', data=df).fit()
Newey = OLS2.get_robustcov_results(cov_type='HAC',maxlags=1)

print(summary_col([OLS, Newey],stars=True,float_format='%0.3f',
                  model_names=['OLS\n(b/se)','Newey\n(b/se)'],
                 info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared), 
                           'Adj.R2':lambda x: "{:.3f}".format(x.rsquared_adj)}))


                 OLS      Newey  
                (b/se)    (b/se) 
---------------------------------
Intercept      -17.803  -6.663***
               (21.045) (1.375)  
R-squared      0.305    0.889    
R-squared Adj. 0.271    0.876    
afdec6         -0.565*           
               (0.286)           
affile6        -0.032            
               (0.264)           
befile6        0.060             
               (0.261)           
lchempi        3.117***          
               (0.479)           
lgas           0.196             
               (0.907)           
lmincov                 -0.212***
                        (0.042)  
lprgnp                  0.285*** 
                        (0.093)  
lrtwex         0.983**           
               (0.400)           
lusgnp                  0.486*   
                        (0.253)  
t                       -0.027***
                        (0.005)  
N              131      38       
R2             0.305    0.889    
Adj.R2       

In [15]:
print(OLSAR1(OLS2 ,drop1=False).summary()) #PW

Rho =  0.5324697530893846
Rho =  0.5796268387566654
Rho =  0.5999046986034846
Rho =  0.6086113183871994
Rho =  0.6123431100391632
Rho =  0.6139413953891576
Rho =  0.6146257121155141
Rho =  0.6149186696926453
Rho =  0.6150440788117247
Rho =  0.6150977626621191
Rho =  0.6151207428725802
Rho =  0.6151305798698998
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.995
Model:                            OLS   Adj. R-squared (uncentered):              0.994
Method:                 Least Squares   F-statistic:                              1336.
Date:                Tue, 02 Jul 2024   Prob (F-statistic):                    4.55e-37
Time:                        18:08:47   Log-Likelihood:                          83.937
No. Observations:                  38   AIC:                                     -157.9
Df Residuals:                      33   BIC:                            

### Example 12.8. Heteroskedasticity and the Efficient Markets Hypothesis

In [16]:
df = dataWoo("nyse")
reg1 = smf.ols('df[("return")] ~ return_1', data=df).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:         df[("return")]   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.399
Date:                Tue, 02 Jul 2024   Prob (F-statistic):              0.122
Time:                        18:08:47   Log-Likelihood:                -1491.2
No. Observations:                 689   AIC:                             2986.
Df Residuals:                     687   BIC:                             2996.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1796      0.081      2.225      0.0

In [17]:
df['u2'] = np.square(reg1.resid)
print(smf.ols('u2 ~ return_1', data=df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                     u2   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     30.05
Date:                Tue, 02 Jul 2024   Prob (F-statistic):           5.90e-08
Time:                        18:08:47   Log-Likelihood:                -2639.9
No. Observations:                 689   AIC:                             5284.
Df Residuals:                     687   BIC:                             5293.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6565      0.428     10.888      0.0

### Example 12.9. ARCH in Stock Returns

In [18]:
df = dataWoo("nyse")
df['u2']= np.square(smf.ols('df[("return")] ~ return_1', data=df).fit().resid)
print(smf.ols('u2 ~ u2.shift(1)', data=df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                     u2   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     87.92
Date:                Tue, 02 Jul 2024   Prob (F-statistic):           9.71e-20
Time:                        18:08:47   Log-Likelihood:                -2609.7
No. Observations:                 688   AIC:                             5223.
Df Residuals:                     686   BIC:                             5233.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.9474      0.440      6.695      

In [19]:
df['u']= smf.ols('df[("return")] ~ return_1', data=df).fit().resid
print(smf.ols('u ~ u.shift(1)', data=df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      u   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                  0.001354
Date:                Tue, 02 Jul 2024   Prob (F-statistic):              0.971
Time:                        18:08:47   Log-Likelihood:                -1489.5
No. Observations:                 688   AIC:                             2983.
Df Residuals:                     686   BIC:                             2992.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0012      0.081     -0.015      0.9